In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import rgb_to_hsv, hsv_to_rgb
from PIL import Image
import math
import pickle
import time
import json
import tensorflow.keras as keras
from keras.layers import Input, Layer, LeakyReLU, BatchNormalization, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Add, Lambda
from keras.models import Model, model_from_json, load_model
from keras.optimizers import SGD, Adam
from keras.regularizers import l2
from keras.callbacks import TerminateOnNaN, ModelCheckpoint, Callback, EarlyStopping
import keras.backend as K
import os
K.clear_session()
K.set_floatx('float32')
print("Running Tensorflow version", tf.__version__)
print("Keras is running on", K.backend(), "backend")

Running Tensorflow version 2.1.0
Keras is running on tensorflow backend


Using TensorFlow backend.


In [2]:
# problem with model with rounding 
# '''     
def roundingAlgo(x):
    return x - x % 1
# '''
'''
def roundingAlgo(x): 
    # simplification of the first algo loop by simplifying the expression for range(1,7)
    # problem - rounding function is still slow = 2,5 hours / epoch
    # all non-speed problem of first algo still applies
    result = x - 0.5
    resultCos = K.cos(2 * math.pi * x)
    return result + K.sin(2 * math.pi * x) * (1 + resultCos) * (13 + 2 * resultCos - 18 * K.pow(resultCos, 2) - 32 * K.pow(resultCos, 3) + 80 * K.pow(resultCos, 4)) / 15
# '''
'''
def roundingAlgo(x): 
    # made to fool the engine to have a gradient
    return 0 * x + K.round(x)
# '''


# check https://github.com/keras-team/keras/issues/2218
# check https://github.com/keras-team/keras/issues/2221
# https://www.tensorflow.org/api_docs/python/tf/custom_gradient
class RoundQinf_4(Layer):
    def __init__(self, **kwargs):
        super(RoundQinf_4, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundQinf_4, self).build(input_shape)
    def call(self, X):
        return roundingAlgo(X * 16) / 16.0
    def get_config(self):
        base_config = super(RoundQinf_4, self).get_config()
        return dict(list(base_config.items()))
class RoundOverflowQ7_12(Layer):
    def __init__(self, **kwargs):
        super(RoundOverflowQ7_12, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundOverflowQ7_12, self).build(input_shape)
    def call(self, X):
        return (((roundingAlgo(X * 4096) + 524288) % 1048576) - 524288) / 4096.0
    def get_config(self):
        base_config = super(RoundOverflowQ7_12, self).get_config()
        return dict(list(base_config.items()))
class RoundQx_8minusx(Layer):
    def __init__(self, **kwargs):
        super(RoundQx_8minusx, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundQx_8minusx, self).build(input_shape)
    def call(self, X):
        pow_2_to_expplus8 = np.exp2(8 + (K.log(K.abs(X)) / math.log(2)) // 1)
        return roundingAlgo(X * pow_2_to_expplus8) / pow_2_to_expplus8
    def get_config(self):
        base_config = super(RoundQx_8minusx, self).get_config()
        return dict(list(base_config.items()))
class RoundOverflowQ3_4(Layer):
    def __init__(self, **kwargs):
        super(RoundOverflowQ3_4, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundOverflowQ3_4, self).build(input_shape)
    def call(self, X):
        return (((roundingAlgo(X * 16) + 128) % 256) - 128) / 16.0
    def get_config(self):
        base_config = super(RoundOverflowQ3_4, self).get_config()
        return dict(list(base_config.items()))
class Identity(Layer):
    def __init__(self, **kwargs):
        super(Identity, self).__init__(**kwargs)
        self.trainable = False
    def call(self, X):
        return X
    def get_config(self):
        base_config = super(Identity, self).get_config()
        return dict(list(base_config.items()))
class IdentityFinalLayer(Layer):
    def __init__(self, **kwargs):
        super(IdentityFinalLayer, self).__init__(**kwargs)
        self.trainable = False
    def call(self, X):
        return X
    def get_config(self):
        base_config = super(IdentityFinalLayer, self).get_config()
        return dict(list(base_config.items()))
    
def DBL(previousLayer, layerFilter, kernelSize=(3, 3), roundingFunction=Identity, name=None):
    placeholder = ""
    if name is not None:
        placeholder = str(name)
    else:
        placeholder = str(time.time_ns())
    return roundingFunction(name="ThirdRound_"+placeholder, dtype=K.floatx())(
        LeakyReLU(alpha=0.1, dtype=K.floatx())(
            roundingFunction(name="SecondRound_"+placeholder, dtype=K.floatx())(
                BatchNormalization(name="BatchNorm_"+placeholder, dtype=K.floatx())(
                    roundingFunction(name="FirstRound_"+placeholder, dtype=K.floatx())(
                        Conv2D(filters=layerFilter, kernel_size=kernelSize, padding='same', use_bias=False, kernel_regularizer=l2(5e-4), name="Conv2D_"+placeholder, dtype=K.floatx())(
                            previousLayer
                        )
                    )
                )
            )
        )
    )
print("Custom layer classes successfully defined")

Custom layer classes successfully defined


In [3]:
classificationClass = 80

print("Number of class classification is", classificationClass)

Number of class classification is 80


In [4]:
print()
def rand(a=0, b=1):
    return np.random.rand()*(b-a) + a

def get_random_data(annotation_line, input_shape, random=True, max_boxes=20, jitter=.3, hue=.1, sat=1.5, val=1.5, proc_img=True):
    '''
    random preprocessing for real-time data augmentation 
    
    random=True induces image processing (better data accuracy with cost of cycles)
    '''
    line = annotation_line.split()
    image = Image.open(line[0])
    iw, ih = image.size
    h, w = input_shape
    box = np.array([np.array(list(map(int,box.split(',')))) for box in line[1:]])

    if not random:
        # resize image
        scale = min(w/iw, h/ih)
        nw = int(iw*scale)
        nh = int(ih*scale)
        dx = (w-nw)//2
        dy = (h-nh)//2
        image_data=0
        if proc_img:
            image = image.resize((nw,nh), Image.BICUBIC)
            new_image = Image.new('RGB', (w,h), (128,128,128))
            new_image.paste(image, (dx, dy))
            image_data = np.array(new_image)/255.

        # correct boxes
        box_data = np.zeros((max_boxes,5))
        if len(box)>0:
            np.random.shuffle(box)
            if len(box)>max_boxes: box = box[:max_boxes]
            box[:, [0,2]] = box[:, [0,2]]*scale + dx
            box[:, [1,3]] = box[:, [1,3]]*scale + dy
            box_data[:len(box)] = box

        return image_data, box_data

    # resize image
    new_ar = w/h * rand(1-jitter,1+jitter)/rand(1-jitter,1+jitter)
    scale = rand(.25, 2)
    if new_ar < 1:
        nh = int(scale*h)
        nw = int(nh*new_ar)
    else:
        nw = int(scale*w)
        nh = int(nw/new_ar)
    image = image.resize((nw,nh), Image.BICUBIC)

    # place image
    dx = int(rand(0, w-nw))
    dy = int(rand(0, h-nh))
    new_image = Image.new('RGB', (w,h), (128,128,128))
    new_image.paste(image, (dx, dy))
    image = new_image

    # flip image or not
    flip = rand()<.5
    if flip: image = image.transpose(Image.FLIP_LEFT_RIGHT)

    # distort image
    hue = rand(-hue, hue)
    sat = rand(1, sat) if rand()<.5 else 1/rand(1, sat)
    val = rand(1, val) if rand()<.5 else 1/rand(1, val)
    x = rgb_to_hsv(np.array(image)/255.)
    x[..., 0] += hue
    x[..., 0][x[..., 0]>1] -= 1
    x[..., 0][x[..., 0]<0] += 1
    x[..., 1] *= sat
    x[..., 2] *= val
    x[x>1] = 1
    x[x<0] = 0
    image_data = hsv_to_rgb(x) # numpy array, 0 to 1

    # correct boxes
    box_data = np.zeros((max_boxes,5))
    if len(box)>0:
        np.random.shuffle(box)
        box[:, [0,2]] = box[:, [0,2]]*nw/iw + dx
        box[:, [1,3]] = box[:, [1,3]]*nh/ih + dy
        if flip: box[:, [0,2]] = w - box[:, [2,0]]
        box[:, 0:2][box[:, 0:2]<0] = 0
        box[:, 2][box[:, 2]>w] = w
        box[:, 3][box[:, 3]>h] = h
        box_w = box[:, 2] - box[:, 0]
        box_h = box[:, 3] - box[:, 1]
        box = box[np.logical_and(box_w>1, box_h>1)] # discard invalid box
        if len(box)>max_boxes: box = box[:max_boxes]
        box_data[:len(box)] = box

    return image_data, box_data

def preprocess_true_boxes(true_boxes, input_shape, anchors, num_classes):
    '''Preprocess true boxes to training input format

    Parameters
    ----------
    true_boxes: array, shape=(m, T, 5)
        Absolute x_min, y_min, x_max, y_max, class_id relative to input_shape.
    input_shape: array-like, hw, multiples of 32
    anchors: array, shape=(N, 2), wh
    num_classes: integer

    Returns
    -------
    y_true: list of array, shape like yolo_outputs, xywh are reletive value

    '''
    assert (true_boxes[..., 4]<num_classes).all(), 'class id must be less than num_classes'
    num_layers = len(anchors)//3 # default setting
    anchor_mask = [[6,7,8], [3,4,5], [0,1,2]] if num_layers==3 else [[3,4,5], [1,2,3]]

    true_boxes = np.array(true_boxes, dtype='float32')
    input_shape = np.array(input_shape, dtype='int32')
    boxes_xy = (true_boxes[..., 0:2] + true_boxes[..., 2:4]) // 2
    boxes_wh = true_boxes[..., 2:4] - true_boxes[..., 0:2]
    true_boxes[..., 0:2] = boxes_xy/input_shape[::-1]
    true_boxes[..., 2:4] = boxes_wh/input_shape[::-1]

    m = true_boxes.shape[0]
    grid_shapes = [input_shape//{0:32, 1:16, 2:8}[l] for l in range(num_layers)]
    y_true = [np.zeros((m,grid_shapes[l][0],grid_shapes[l][1],len(anchor_mask[l]),5+num_classes),
        dtype='float32') for l in range(num_layers)]

    # Expand dim to apply broadcasting.
    anchors = np.expand_dims(anchors, 0)
    anchor_maxes = anchors / 2.
    anchor_mins = -anchor_maxes
    valid_mask = boxes_wh[..., 0]>0

    for b in range(m):
        # Discard zero rows.
        wh = boxes_wh[b, valid_mask[b]]
        if len(wh)==0: continue
        # Expand dim to apply broadcasting.
        wh = np.expand_dims(wh, -2)
        box_maxes = wh / 2.
        box_mins = -box_maxes

        intersect_mins = np.maximum(box_mins, anchor_mins)
        intersect_maxes = np.minimum(box_maxes, anchor_maxes)
        intersect_wh = np.maximum(intersect_maxes - intersect_mins, 0.)
        intersect_area = intersect_wh[..., 0] * intersect_wh[..., 1]
        box_area = wh[..., 0] * wh[..., 1]
        anchor_area = anchors[..., 0] * anchors[..., 1]
        iou = intersect_area / (box_area + anchor_area - intersect_area)

        # Find best anchor for each true box
        best_anchor = np.argmax(iou, axis=-1)

        for t, n in enumerate(best_anchor):
            for l in range(num_layers):
                if n in anchor_mask[l]:
                    i = np.floor(true_boxes[b,t,0]*grid_shapes[l][1]).astype('int32')
                    j = np.floor(true_boxes[b,t,1]*grid_shapes[l][0]).astype('int32')
                    k = anchor_mask[l].index(n)
                    c = true_boxes[b,t, 4].astype('int32')
                    y_true[l][b, j, i, k, 0:4] = true_boxes[b,t, 0:4]
                    y_true[l][b, j, i, k, 4] = 1
                    y_true[l][b, j, i, k, 5+c] = 1

    return y_true

def data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes):
    '''data generator for fit_generator'''
    n = len(annotation_lines)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
            if i==0:
                np.random.shuffle(annotation_lines)
            image, box = get_random_data(annotation_lines[i], input_shape, random=True)
            image_data.append(image)
            box_data.append(box)
            i = (i + 1) % n
        image_data = np.array(image_data)
        box_data = np.array(box_data)
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)
        yield [image_data, *y_true], np.zeros(batch_size)

def data_generator_wrapper(annotation_lines, batch_size, input_shape, anchors, num_classes):
    n = len(annotation_lines)
    if n==0 or batch_size<=0: return None
    return data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes)
print("Target data generator successfully defined")


Target data generator successfully defined


In [5]:
# diambil dari https://github.com/qqwweee/keras-yolo3
'''
MIT License

Copyright (c) 2018 qqwweee

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
'''

image_size = (448,448)
image_height, image_width = image_size

def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

tinyYolo_anchors = get_anchors("../CNN-VLSI/tiny_yolo_anchors.txt")

def yolo_head(feats, anchors, num_classes, input_shape, calc_loss=False):
    """Convert final layer features to bounding box parameters."""
    num_anchors = len(anchors)
    # Reshape to batch, height, width, num_anchors, box_params.
    anchors_tensor = K.reshape(K.constant(anchors), [1, 1, 1, num_anchors, 2])

    grid_shape = K.shape(feats)[1:3] # height, width
    grid_y = K.tile(K.reshape(K.arange(0, stop=grid_shape[0]), [-1, 1, 1, 1]),
        [1, grid_shape[1], 1, 1])
    grid_x = K.tile(K.reshape(K.arange(0, stop=grid_shape[1]), [1, -1, 1, 1]),
        [grid_shape[0], 1, 1, 1])
    grid = K.concatenate([grid_x, grid_y])
    grid = K.cast(grid, K.dtype(feats))

    feats = K.reshape(
        feats, [-1, grid_shape[0], grid_shape[1], num_anchors, num_classes + 5])

    # Adjust preditions to each spatial grid point and anchor size.
    # edited by instructions in https://stackoverflow.com/questions/57558476/training-a-keras-model-yields-multiple-optimizer-errors
    box_xy = (K.sigmoid(feats[..., :2]) + grid) / K.cast(grid_shape[...,::-1], K.dtype(feats))
    box_wh = K.exp(feats[..., 2:4]) * anchors_tensor / K.cast(input_shape[...,::-1], K.dtype(feats))
    box_confidence = K.sigmoid(feats[..., 4:5])
    box_class_probs = K.sigmoid(feats[..., 5:])

    if calc_loss == True:
        return grid, feats, box_xy, box_wh
    return box_xy, box_wh, box_confidence, box_class_probs

def box_iou(b1, b2):
    '''Return iou tensor

    Parameters
    ----------
    b1: tensor, shape=(i1,...,iN, 4), xywh
    b2: tensor, shape=(j, 4), xywh

    Returns
    -------
    iou: tensor, shape=(i1,...,iN, j)

    '''

    # Expand dim to apply broadcasting.
    b1 = K.expand_dims(b1, -2)
    b1_xy = b1[..., :2]
    b1_wh = b1[..., 2:4]
    b1_wh_half = b1_wh/2.
    b1_mins = b1_xy - b1_wh_half
    b1_maxes = b1_xy + b1_wh_half

    # Expand dim to apply broadcasting.
    b2 = K.expand_dims(b2, 0)
    b2_xy = b2[..., :2]
    b2_wh = b2[..., 2:4]
    b2_wh_half = b2_wh/2.
    b2_mins = b2_xy - b2_wh_half
    b2_maxes = b2_xy + b2_wh_half

    intersect_mins = K.maximum(b1_mins, b2_mins)
    intersect_maxes = K.minimum(b1_maxes, b2_maxes)
    intersect_wh = K.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_area = intersect_wh[..., 0] * intersect_wh[..., 1]
    b1_area = b1_wh[..., 0] * b1_wh[..., 1]
    b2_area = b2_wh[..., 0] * b2_wh[..., 1]
    iou = intersect_area / (b1_area + b2_area - intersect_area)

    return iou

def yolo_loss(args, anchors, num_classes, ignore_thresh=.5, print_loss=False):
    '''Return yolo_loss tensor

    Parameters
    ----------
    yolo_outputs: list of tensor, the output of yolo_body or tiny_yolo_body
    y_true: list of array, the output of preprocess_true_boxes
    anchors: array, shape=(N, 2), wh
    num_classes: integer
    ignore_thresh: float, the iou threshold whether to ignore object confidence loss

    Returns
    -------
    loss: tensor, shape=(1,)

    '''
    num_layers = len(anchors)//3 # default setting
    yolo_outputs = args[:num_layers]
    y_true = args[num_layers:]
    anchor_mask = [[6,7,8], [3,4,5], [0,1,2]] if num_layers==3 else [[3,4,5], [1,2,3]]
    input_shape = K.cast(K.shape(yolo_outputs[0])[1:3] * 32, K.dtype(y_true[0]))
    grid_shapes = [K.cast(K.shape(yolo_outputs[l])[1:3], K.dtype(y_true[0])) for l in range(num_layers)]
    loss = 0
    m = K.shape(yolo_outputs[0])[0] # batch size, tensor
    mf = K.cast(m, K.dtype(yolo_outputs[0]))

    for l in range(num_layers):
        object_mask = y_true[l][..., 4:5]
        true_class_probs = y_true[l][..., 5:]

        grid, raw_pred, pred_xy, pred_wh = yolo_head(yolo_outputs[l], anchors[anchor_mask[l]], num_classes, input_shape, calc_loss=True)
        pred_box = K.concatenate([pred_xy, pred_wh])

        # Darknet raw box to calculate loss.
        raw_true_xy = y_true[l][..., :2]*grid_shapes[l][::-1] - grid
        raw_true_wh = K.log(y_true[l][..., 2:4] / anchors[anchor_mask[l]] * input_shape[::-1] + 1e-10)
        raw_true_wh = K.switch(object_mask, raw_true_wh, K.zeros_like(raw_true_wh)) # avoid log(0)=-inf
        box_loss_scale = 2 - y_true[l][...,2:3]*y_true[l][...,3:4]

        # Find ignore mask, iterate over each of batch.
        ignore_mask = tf.TensorArray(K.dtype(y_true[0]), size=1, dynamic_size=True)
        object_mask_bool = K.cast(object_mask, 'bool')
        def loop_body(b, ignore_mask):
            true_box = tf.boolean_mask(y_true[l][b,...,0:4], object_mask_bool[b,...,0])
            iou = box_iou(pred_box[b], true_box)
            best_iou = K.max(iou, axis=-1)
            ignore_mask = ignore_mask.write(b, K.cast(best_iou<ignore_thresh, K.dtype(true_box)))
            return b+1, ignore_mask
        _, ignore_mask = tf.while_loop(lambda b,*args: b<m, loop_body, [0, ignore_mask])
        ignore_mask = ignore_mask.stack()
        ignore_mask = K.expand_dims(ignore_mask, -1)

        # K.binary_crossentropy is helpful to avoid exp overflow.
        xy_loss = object_mask * box_loss_scale * K.binary_crossentropy(raw_true_xy, raw_pred[...,0:2], from_logits=True)
        wh_loss = object_mask * box_loss_scale * 0.5 * K.square(raw_true_wh-raw_pred[...,2:4])
        confidence_loss = object_mask * K.binary_crossentropy(object_mask, raw_pred[...,4:5], from_logits=True)+ \
            (1-object_mask) * K.binary_crossentropy(object_mask, raw_pred[...,4:5], from_logits=True) * ignore_mask
        class_loss = object_mask * K.binary_crossentropy(true_class_probs, raw_pred[...,5:], from_logits=True)

        xy_loss = K.sum(xy_loss) / mf
        wh_loss = K.sum(wh_loss) / mf
        confidence_loss = K.sum(confidence_loss) / mf
        class_loss = K.sum(class_loss) / mf
        loss += xy_loss + wh_loss + confidence_loss + class_loss
    return loss

In [6]:
with open("../CNN-VLSI/train.txt") as trainText:
    train_annotation_lines = trainText.readlines()
with open("../CNN-VLSI/val.txt") as valText:
    val_annotation_lines = valText.readlines()
lenTrain = len(train_annotation_lines)
print("# of training data is", lenTrain)
lenVal = len(val_annotation_lines)
print("# of validation data is", lenVal)
np.random.shuffle(train_annotation_lines)
np.random.shuffle(val_annotation_lines)

# of training data is 117266
# of validation data is 4952


In [7]:
# model_0 does no rounding (float32 operation)

In [8]:
model_0 = None
# '''
model_0_input = Input(shape=(None, None, 3), name="model_0_inputLayer")
# model_0_pointer = model_0_input
print("Input shape:", model_0_input.shape) # 448 x 448 x 3
model_0_startBranch = DBL(roundingFunction=Identity, previousLayer=model_0_input, layerFilter=16, name="model_0_layer0_branch") 
# model_0_pointer = model_0_startBranch
# print(model_0_pointer.shape) # 448 x 448 x 16
model_0_startBranch = MaxPooling2D(pool_size=(2, 2))(model_0_startBranch)
# model_0_pointer = model_0_startBranch
# print(model_0_pointer.shape) # 224 x 224 x 16
model_0_startBranch = DBL(roundingFunction=Identity, previousLayer=model_0_startBranch, layerFilter=32, name="model_0_layer1_branch")
# model_0_pointer = model_0_startBranch
# print(model_0_pointer.shape) # 224 x 224 x 32
model_0_startBranch = MaxPooling2D(pool_size=(2, 2))(model_0_startBranch)
# model_0_pointer = model_0_startBranch
# print(model_0_pointer.shape) # 112 x 112 x 32
model_0_startBranch = DBL(roundingFunction=Identity, previousLayer=model_0_startBranch, layerFilter=64, name="model_0_layer2_branch")
# model_0_pointer = model_0_startBranch
# print(model_0_pointer.shape) # 112 x 112 x 64
model_0_startBranch = MaxPooling2D(pool_size=(2, 2))(model_0_startBranch)
# model_0_pointer = model_0_startBranch
# print(model_0_pointer.shape) # 56 x 56 x 64
model_0_startBranch = DBL(roundingFunction=Identity, previousLayer=model_0_startBranch, layerFilter=128, name="model_0_layer3_branch")
# model_0_pointer = model_0_startBranch
# print(model_0_pointer.shape) # 56 x 56 x 128
model_0_startBranch = MaxPooling2D(pool_size=(2, 2))(model_0_startBranch)
# model_0_pointer = model_0_startBranch
# print(model_0_pointer.shape) # 28 x 28 x 128
model_0_startBranch = DBL(roundingFunction=Identity, previousLayer=model_0_startBranch, layerFilter=256, name="model_0_layer4_branch")
# model_0_pointer = model_0_startBranch
# print(model_0_pointer.shape) # 28 x 28 x 256
print("Branch split from main branch - following branch 0") # 2 branch split from startBranch (28 x 28 x 256), following model_0_branch0
model_0_branch0 = MaxPooling2D(pool_size=(2, 2))(model_0_startBranch)
# model_0_pointer = model_0_branch0
# print(model_0_pointer.shape) # 14 x 14 x 256
model_0_branch0 = DBL(roundingFunction=Identity, previousLayer=model_0_branch0, layerFilter=512, name="model_0_layer5_branch0")
# model_0_pointer = model_0_branch0
# print(model_0_pointer.shape) # 14 x 14 x 512
model_0_branch0 = MaxPooling2D(pool_size=(2, 2), strides=1, padding='same')(model_0_branch0)
# model_0_pointer = model_0_branch0
# print(model_0_pointer.shape) # 14 x 14 x 512
model_0_branch0 = DBL(roundingFunction=Identity, previousLayer=model_0_branch0, layerFilter=1024, name="model_0_layer6_branch0")
# model_0_pointer = model_0_branch0
# print(model_0_pointer.shape) # 14 x 14 x 1024
model_0_branch0 = DBL(roundingFunction=Identity, previousLayer=model_0_branch0, layerFilter=256, kernelSize=(1, 1), name="model_0_layer7_branch0")
# model_0_pointer = model_0_branch0
# print(model_0_pointer.shape) # 14 x 14 x 256
print("Branch split from branch 0 - following branch 0,0") # 2 branch split from model_0_branch0 (14 x 14 x 256), following model_0_branch00
model_0_branch00 = DBL(roundingFunction=Identity, previousLayer=model_0_branch0, layerFilter=128, kernelSize=(1, 1), name="model_0_layer8_branch00")
# model_0_pointer = model_0_branch00
# print(model_0_pointer.shape) # 14 x 14 x 128
model_0_branch00 = UpSampling2D()(model_0_branch00)
# model_0_pointer = model_0_branch00
# print(model_0_pointer.shape) # 28 x 28 x 128
print("Branch merge from branch 1 and branch 0,0") # 2 branch merge from model_0_branch1 (unchanged from model_0_startBranch) and model_0_branch00
model_0_mergedBranch = Concatenate()([model_0_startBranch, model_0_branch00])
# model_0_pointer = model_0_mergedBranch
# print(model_0_pointer.shape) # 28 x 28 x 384
model_0_mergedBranch = DBL(roundingFunction=Identity, previousLayer=model_0_mergedBranch, layerFilter=256, name="model_0_layer9_branch1")
# model_0_pointer = model_0_mergedBranch
# print(model_0_pointer.shape) # 28 x 28 x 256
model_0_mergedBranch = DBL(roundingFunction=Identity, previousLayer=model_0_mergedBranch, layerFilter=3 * (4 + 1 + classificationClass), name="model_0_layerA_branch1")
# model_0_pointer = model_0_mergedBranch
print("Model output 1 shape:", model_0_mergedBranch.shape) # 28 x 28 x (3 * (5 + classificationClass))
model_0_mergedBranch = IdentityFinalLayer(name="model_0_outputLayer_1")(model_0_mergedBranch)
print() # OUTPUT = model_0_mergedBranch (note: 26 x 26 grid untuk deteksi objek kecil)

print("Branch split from branch 0 - following branch 0,1")# following model_0_branch01
model_0_branch01 = DBL(roundingFunction=Identity, previousLayer=model_0_branch0, layerFilter=512, name="model_0_layer8_branch01")
# model_0_pointer = model_0_branch01
# print(model_0_pointer.shape) # 14 x 14 x 512
model_0_branch01 = DBL(roundingFunction=Identity, previousLayer=model_0_branch01, layerFilter=3 * (4 + 1 + classificationClass), name="model_0_layer9_branch01")
# model_0_pointer = model_0_branch01
print("Model output 0 shape:", model_0_branch01.shape) # 14 x 14 x (3 * (5 + classificationClass))
model_0_branch01 = IdentityFinalLayer(name="model_0_outputLayer_0")(model_0_branch01)
print() # OUTPUT = model_0_branch01 (note: 13 x 13 grid untuk deteksi objek besar)

model_0_actual = Model(inputs=model_0_input, outputs=[model_0_branch01, model_0_mergedBranch]) # mengikuti model dari https://github.com/awe777/keras-yolo3/blob/master/yolo3/model.py

with open("./saved_models/model_0_inferenceModel.json", "wt") as jsonFile:
    jsonFile.write(model_0_actual.to_json())

try:
    model_0_actual.load_weights("../keras-yolo3/model_data/yolov3_tiny-model_0.h5", by_name=True, skip_mismatch=True)
#     model_0_actual.load_weights("./saved_models/model_0_checkpoint.h5", by_name=True, skip_mismatch=True)
    print("Weight load attempt success for model_0")
except Exception as e:
    print("Failed to load existing model for model_0:", e)
try:
    model_0_actual.save_weights("./saved_models/model_0_trainModel.h5")
    print("Loaded model is successfully re-saved")
except Exception as e:
    print("Failed to save loaded model:", e)
# added code from https://github.com/awe777/keras-yolo3/blob/master/yolo3/model.py
setArgs_model_0 = {
    'anchors': tinyYolo_anchors, 
    'num_classes': classificationClass, 
    'ignore_thresh': 0.2
}

y_true_model_0 = [Input(shape=(image_height//{0:32, 1:16}[l], image_width//{0:32, 1:16}[l], 3, classificationClass + 5)) for l in range(2)]

model_0_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss_model_0', arguments=setArgs_model_0)([*model_0_actual.output, *y_true_model_0])

model_0 = Model([model_0_actual.input, *y_true_model_0], model_0_loss)

# model_0_actual adalah working model yang akan dipakai untuk inference
# y_true adalah layer input yang akan diisikan nilai sebenarnya
# cara training model_0 adalah pendekatan sehingga output model_0 sedekat mungkin dengan 0 (model_0 ≈ model_0_actual - y_true)
model_0_learnRate = 1e-2
model_0.compile(optimizer=Adam(lr=model_0_learnRate), loss={'yolo_loss_model_0': lambda y_true, y_pred: y_pred})

# loss adalah seberapa jauh nilai perbedaan output dengan 0

print("Model model_0 compilation complete") # tinggal porting line 54 sampai 61 dari https://github.com/awe777/keras-yolo3/blob/master/train.py, dilakukan di cell training di bawah
# '''
print()

Input shape: (None, None, None, 3)
Branch split from main branch - following branch 0
Branch split from branch 0 - following branch 0,0
Branch merge from branch 1 and branch 0,0
Model output 1 shape: (None, None, None, 255)

Branch split from branch 0 - following branch 0,1
Model output 0 shape: (None, None, None, 255)

Weight load attempt success for model_0
Loaded model is successfully re-saved
Model model_0 compilation complete



In [9]:
# model_1 rounds output to multiples of 1/16
# otherwise has float32 ranges

In [10]:
model_1 = None
# '''
model_1_input = Input(shape=(None, None, 3), name="model_1_inputLayer")
# model_1_pointer = model_1_input
print("Input shape:", model_1_input.shape) # 448 x 448 x 3
model_1_startBranch = DBL(roundingFunction=RoundQinf_4, previousLayer=model_1_input, layerFilter=16, name="model_1_layer0_branch") 
# model_1_pointer = model_1_startBranch
# print(model_1_pointer.shape) # 448 x 448 x 16
model_1_startBranch = MaxPooling2D(pool_size=(2, 2))(model_1_startBranch)
# model_1_pointer = model_1_startBranch
# print(model_1_pointer.shape) # 224 x 224 x 16
model_1_startBranch = DBL(roundingFunction=RoundQinf_4, previousLayer=model_1_startBranch, layerFilter=32, name="model_1_layer1_branch")
# model_1_pointer = model_1_startBranch
# print(model_1_pointer.shape) # 224 x 224 x 32
model_1_startBranch = MaxPooling2D(pool_size=(2, 2))(model_1_startBranch)
# model_1_pointer = model_1_startBranch
# print(model_1_pointer.shape) # 112 x 112 x 32
model_1_startBranch = DBL(roundingFunction=RoundQinf_4, previousLayer=model_1_startBranch, layerFilter=64, name="model_1_layer2_branch")
# model_1_pointer = model_1_startBranch
# print(model_1_pointer.shape) # 112 x 112 x 64
model_1_startBranch = MaxPooling2D(pool_size=(2, 2))(model_1_startBranch)
# model_1_pointer = model_1_startBranch
# print(model_1_pointer.shape) # 56 x 56 x 64
model_1_startBranch = DBL(roundingFunction=RoundQinf_4, previousLayer=model_1_startBranch, layerFilter=128, name="model_1_layer3_branch")
# model_1_pointer = model_1_startBranch
# print(model_1_pointer.shape) # 56 x 56 x 128
model_1_startBranch = MaxPooling2D(pool_size=(2, 2))(model_1_startBranch)
# model_1_pointer = model_1_startBranch
# print(model_1_pointer.shape) # 28 x 28 x 128
model_1_startBranch = DBL(roundingFunction=RoundQinf_4, previousLayer=model_1_startBranch, layerFilter=256, name="model_1_layer4_branch")
# model_1_pointer = model_1_startBranch
# print(model_1_pointer.shape) # 28 x 28 x 256
print("Branch split from main branch - following branch 0") # 2 branch split from startBranch (28 x 28 x 256), following model_1_branch0
model_1_branch0 = MaxPooling2D(pool_size=(2, 2))(model_1_startBranch)
# model_1_pointer = model_1_branch0
# print(model_1_pointer.shape) # 14 x 14 x 256
model_1_branch0 = DBL(roundingFunction=RoundQinf_4, previousLayer=model_1_branch0, layerFilter=512, name="model_1_layer5_branch0")
# model_1_pointer = model_1_branch0
# print(model_1_pointer.shape) # 14 x 14 x 512
model_1_branch0 = MaxPooling2D(pool_size=(2, 2), strides=1, padding='same')(model_1_branch0)
# model_1_pointer = model_1_branch0
# print(model_1_pointer.shape) # 14 x 14 x 512
model_1_branch0 = DBL(roundingFunction=RoundQinf_4, previousLayer=model_1_branch0, layerFilter=1024, name="model_1_layer6_branch0")
# model_1_pointer = model_1_branch0
# print(model_1_pointer.shape) # 14 x 14 x 1024
model_1_branch0 = DBL(roundingFunction=RoundQinf_4, previousLayer=model_1_branch0, layerFilter=256, kernelSize=(1, 1), name="model_1_layer7_branch0")
# model_1_pointer = model_1_branch0
# print(model_1_pointer.shape) # 14 x 14 x 256
print("Branch split from branch 0 - following branch 0,0") # 2 branch split from model_1_branch0 (14 x 14 x 256), following model_1_branch00
model_1_branch00 = DBL(roundingFunction=RoundQinf_4, previousLayer=model_1_branch0, layerFilter=128, kernelSize=(1, 1), name="model_1_layer8_branch00")
# model_1_pointer = model_1_branch00
# print(model_1_pointer.shape) # 14 x 14 x 128
model_1_branch00 = UpSampling2D()(model_1_branch00)
# model_1_pointer = model_1_branch00
# print(model_1_pointer.shape) # 28 x 28 x 128
print("Branch merge from branch 1 and branch 0,0") # 2 branch merge from model_1_branch1 (unchanged from model_1_startBranch) and model_1_branch00
model_1_mergedBranch = Concatenate()([model_1_startBranch, model_1_branch00])
# model_1_pointer = model_1_mergedBranch
# print(model_1_pointer.shape) # 28 x 28 x 384
model_1_mergedBranch = DBL(roundingFunction=RoundQinf_4, previousLayer=model_1_mergedBranch, layerFilter=256, name="model_1_layer9_branch1")
# model_1_pointer = model_1_mergedBranch
# print(model_1_pointer.shape) # 28 x 28 x 256
model_1_mergedBranch = DBL(roundingFunction=RoundQinf_4, previousLayer=model_1_mergedBranch, layerFilter=3 * (4 + 1 + classificationClass), name="model_1_layerA_branch1")
# model_1_pointer = model_1_mergedBranch
print("Model output 1 shape:", model_1_mergedBranch.shape) # 28 x 28 x (3 * (5 + classificationClass))
model_1_mergedBranch = IdentityFinalLayer(name="model_1_outputLayer_1")(model_1_mergedBranch)
print() # OUTPUT = model_1_mergedBranch (note: 26 x 26 grid untuk deteksi objek kecil)

print("Branch split from branch 0 - following branch 0,1")# following model_1_branch01
model_1_branch01 = DBL(roundingFunction=RoundQinf_4, previousLayer=model_1_branch0, layerFilter=512, name="model_1_layer8_branch01")
# model_1_pointer = model_1_branch01
# print(model_1_pointer.shape) # 14 x 14 x 512
model_1_branch01 = DBL(roundingFunction=RoundQinf_4, previousLayer=model_1_branch01, layerFilter=3 * (4 + 1 + classificationClass), name="model_1_layer9_branch01")
# model_1_pointer = model_1_branch01
print("Model output 0 shape:", model_1_branch01.shape) # 14 x 14 x (3 * (5 + classificationClass))
model_1_branch01 = IdentityFinalLayer(name="model_1_outputLayer_0")(model_1_branch01)
print() # OUTPUT = model_1_branch01 (note: 13 x 13 grid untuk deteksi objek besar)

model_1_actual = Model(inputs=model_1_input, outputs=[model_1_branch01, model_1_mergedBranch]) # mengikuti model dari https://github.com/awe777/keras-yolo3/blob/master/yolo3/model.py

with open("./saved_models/model_1_inferenceModel.json", "wt") as jsonFile:
    jsonFile.write(model_1_actual.to_json())

try:
#     model_1_actual.load_weights("../keras-yolo3/model_data/yolov3_tiny_roundOverflowQ3_4.h5", by_name=True, skip_mismatch=True)
#     model_1_actual.load_weights("./saved_models/model_1_checkpoint.h5", by_name=True, skip_mismatch=True)
    model_1_actual.load_weights("./saved_models/model_0_trainModel.h5", by_name=False, skip_mismatch=True)
    print("Weight load attempt success for model_1")
except Exception as e:
    print("Failed to load existing model for model_1:", e)
try:
    model_1_actual.save_weights("./saved_models/model_1_trainModel.h5")
    print("Loaded model is successfully re-saved")
except Exception as e:
    print("Failed to save loaded model:", e)
# added code from https://github.com/awe777/keras-yolo3/blob/master/yolo3/model.py
setArgs_model_1 = {
    'anchors': tinyYolo_anchors, 
    'num_classes': classificationClass, 
    'ignore_thresh': 0.2
}

y_true_model_1 = [Input(shape=(image_height//{0:32, 1:16}[l], image_width//{0:32, 1:16}[l], 3, classificationClass + 5)) for l in range(2)]

model_1_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss_model_1', arguments=setArgs_model_1)([*model_1_actual.output, *y_true_model_1])

model_1 = Model([model_1_actual.input, *y_true_model_1], model_1_loss)

# model_1_actual adalah working model yang akan dipakai untuk inference
# y_true adalah layer input yang akan diisikan nilai sebenarnya
# cara training model_1 adalah pendekatan sehingga output model_1 sedekat mungkin dengan 0 (model_1 ≈ model_1_actual - y_true)
model_1_learnRate = 1e-2
model_1.compile(optimizer=Adam(lr=model_1_learnRate), loss={'yolo_loss_model_1': lambda y_true, y_pred: y_pred})

# loss adalah seberapa jauh nilai perbedaan output dengan 0

print("Model model_1 compilation complete") # tinggal porting line 54 sampai 61 dari https://github.com/awe777/keras-yolo3/blob/master/train.py, dilakukan di cell training di bawah
# '''
print()

In [11]:
# model_2 approximates Q7.12 signed fixed point operations with integer rules (overflow = positive -> negative & vice versa) 
# Done by rounding to the nearest 1/4096 and capping at [-128, 128) after batch normalization and activation layers

In [12]:
model_2 = None
'''
model_2_input = Input(shape=(None, None, 3), name="model_2_inputLayer")
# model_2_pointer = model_2_input
print("Input shape:", model_2_input.shape) # 448 x 448 x 3
model_2_startBranch = DBL(roundingFunction=RoundOverflowQ7_12, previousLayer=model_2_input, layerFilter=16, name="model_2_layer0_branch") 
# model_2_pointer = model_2_startBranch
# print(model_2_pointer.shape) # 448 x 448 x 16
model_2_startBranch = MaxPooling2D(pool_size=(2, 2))(model_2_startBranch)
# model_2_pointer = model_2_startBranch
# print(model_2_pointer.shape) # 224 x 224 x 16
model_2_startBranch = DBL(roundingFunction=RoundOverflowQ7_12, previousLayer=model_2_startBranch, layerFilter=32, name="model_2_layer1_branch")
# model_2_pointer = model_2_startBranch
# print(model_2_pointer.shape) # 224 x 224 x 32
model_2_startBranch = MaxPooling2D(pool_size=(2, 2))(model_2_startBranch)
# model_2_pointer = model_2_startBranch
# print(model_2_pointer.shape) # 112 x 112 x 32
model_2_startBranch = DBL(roundingFunction=RoundOverflowQ7_12, previousLayer=model_2_startBranch, layerFilter=64, name="model_2_layer2_branch")
# model_2_pointer = model_2_startBranch
# print(model_2_pointer.shape) # 112 x 112 x 64
model_2_startBranch = MaxPooling2D(pool_size=(2, 2))(model_2_startBranch)
# model_2_pointer = model_2_startBranch
# print(model_2_pointer.shape) # 56 x 56 x 64
model_2_startBranch = DBL(roundingFunction=RoundOverflowQ7_12, previousLayer=model_2_startBranch, layerFilter=128, name="model_2_layer3_branch")
# model_2_pointer = model_2_startBranch
# print(model_2_pointer.shape) # 56 x 56 x 128
model_2_startBranch = MaxPooling2D(pool_size=(2, 2))(model_2_startBranch)
# model_2_pointer = model_2_startBranch
# print(model_2_pointer.shape) # 28 x 28 x 128
model_2_startBranch = DBL(roundingFunction=RoundOverflowQ7_12, previousLayer=model_2_startBranch, layerFilter=256, name="model_2_layer4_branch")
# model_2_pointer = model_2_startBranch
# print(model_2_pointer.shape) # 28 x 28 x 256
print("Branch split from main branch - following branch 0") # 2 branch split from startBranch (28 x 28 x 256), following model_2_branch0
model_2_branch0 = MaxPooling2D(pool_size=(2, 2))(model_2_startBranch)
# model_2_pointer = model_2_branch0
# print(model_2_pointer.shape) # 14 x 14 x 256
model_2_branch0 = DBL(roundingFunction=RoundOverflowQ7_12, previousLayer=model_2_branch0, layerFilter=512, name="model_2_layer5_branch0")
# model_2_pointer = model_2_branch0
# print(model_2_pointer.shape) # 14 x 14 x 512
model_2_branch0 = MaxPooling2D(pool_size=(2, 2), strides=1, padding='same')(model_2_branch0)
# model_2_pointer = model_2_branch0
# print(model_2_pointer.shape) # 14 x 14 x 512
model_2_branch0 = DBL(roundingFunction=RoundOverflowQ7_12, previousLayer=model_2_branch0, layerFilter=1024, name="model_2_layer6_branch0")
# model_2_pointer = model_2_branch0
# print(model_2_pointer.shape) # 14 x 14 x 1024
model_2_branch0 = DBL(roundingFunction=RoundOverflowQ7_12, previousLayer=model_2_branch0, layerFilter=256, kernelSize=(1, 1), name="model_2_layer7_branch0")
# model_2_pointer = model_2_branch0
# print(model_2_pointer.shape) # 14 x 14 x 256
print("Branch split from branch 0 - following branch 0,0") # 2 branch split from model_2_branch0 (14 x 14 x 256), following model_2_branch00
model_2_branch00 = DBL(roundingFunction=RoundOverflowQ7_12, previousLayer=model_2_branch0, layerFilter=128, kernelSize=(1, 1), name="model_2_layer8_branch00")
# model_2_pointer = model_2_branch00
# print(model_2_pointer.shape) # 14 x 14 x 128
model_2_branch00 = UpSampling2D()(model_2_branch00)
# model_2_pointer = model_2_branch00
# print(model_2_pointer.shape) # 28 x 28 x 128
print("Branch merge from branch 1 and branch 0,0") # 2 branch merge from model_2_branch1 (unchanged from model_2_startBranch) and model_2_branch00
model_2_mergedBranch = Concatenate()([model_2_startBranch, model_2_branch00])
# model_2_pointer = model_2_mergedBranch
# print(model_2_pointer.shape) # 28 x 28 x 384
model_2_mergedBranch = DBL(roundingFunction=RoundOverflowQ7_12, previousLayer=model_2_mergedBranch, layerFilter=256, name="model_2_layer9_branch1")
# model_2_pointer = model_2_mergedBranch
# print(model_2_pointer.shape) # 28 x 28 x 256
model_2_mergedBranch = DBL(roundingFunction=RoundOverflowQ7_12, previousLayer=model_2_mergedBranch, layerFilter=3 * (4 + 1 + classificationClass), name="model_2_layerA_branch1")
# model_2_pointer = model_2_mergedBranch
print("Model output 1 shape:", model_2_mergedBranch.shape) # 28 x 28 x (3 * (5 + classificationClass))
model_2_mergedBranch = IdentityFinalLayer(name="model_2_outputLayer_1")(model_2_mergedBranch)
print() # OUTPUT = model_2_mergedBranch (note: 26 x 26 grid untuk deteksi objek kecil)

print("Branch split from branch 0 - following branch 0,1")# following model_2_branch01
model_2_branch01 = DBL(roundingFunction=RoundOverflowQ7_12, previousLayer=model_2_branch0, layerFilter=512, name="model_2_layer8_branch01")
# model_2_pointer = model_2_branch01
# print(model_2_pointer.shape) # 14 x 14 x 512
model_2_branch01 = DBL(roundingFunction=RoundOverflowQ7_12, previousLayer=model_2_branch01, layerFilter=3 * (4 + 1 + classificationClass), name="model_2_layer9_branch01")
# model_2_pointer = model_2_branch01
print("Model output 0 shape:", model_2_branch01.shape) # 14 x 14 x (3 * (5 + classificationClass))
model_2_branch01 = IdentityFinalLayer(name="model_2_outputLayer_0")(model_2_branch01)
print() # OUTPUT = model_2_branch01 (note: 13 x 13 grid untuk deteksi objek besar)

model_2_actual = Model(inputs=model_2_input, outputs=[model_2_branch01, model_2_mergedBranch]) # mengikuti model dari https://github.com/awe777/keras-yolo3/blob/master/yolo3/model.py

with open("./saved_models/model_2_inferenceModel.json", "wt") as jsonFile:
    jsonFile.write(model_2_actual.to_json())

try:
#     model_2_actual.load_weights("../keras-yolo3/model_data/yolov3_tiny_roundOverflowQ3_4.h5", by_name=True, skip_mismatch=True)
#     model_2_actual.load_weights("./saved_models/model_2_checkpoint.h5", by_name=True, skip_mismatch=True)
    model_2_actual.load_weights("./saved_models/model_0_trainModel.h5", by_name=False, skip_mismatch=True)
    print("Weight load attempt success for model_2")
except Exception as e:
    print("Failed to load existing model for model_2:", e)
try:
    model_2_actual.save_weights("./saved_models/model_2_trainModel.h5")
    print("Loaded model is successfully re-saved")
except Exception as e:
    print("Failed to save loaded model:", e)
# added code from https://github.com/awe777/keras-yolo3/blob/master/yolo3/model.py
setArgs_model_2 = {
    'anchors': tinyYolo_anchors, 
    'num_classes': classificationClass, 
    'ignore_thresh': 0.2
}

y_true_model_2 = [Input(shape=(image_height//{0:32, 1:16}[l], image_width//{0:32, 1:16}[l], 3, classificationClass + 5)) for l in range(2)]

model_2_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss_model_2', arguments=setArgs_model_2)([*model_2_actual.output, *y_true_model_2])

model_2 = Model([model_2_actual.input, *y_true_model_2], model_2_loss)

# model_2_actual adalah working model yang akan dipakai untuk inference
# y_true adalah layer input yang akan diisikan nilai sebenarnya
# cara training model_2 adalah pendekatan sehingga output model_2 sedekat mungkin dengan 0 (model_2 ≈ model_2_actual - y_true)
model_2_learnRate = 1e-2
model_2.compile(optimizer=Adam(lr=model_2_learnRate), loss={'yolo_loss_model_2': lambda y_true, y_pred: y_pred})

# loss adalah seberapa jauh nilai perbedaan output dengan 0

print("Model model_2 compilation complete") # tinggal porting line 54 sampai 61 dari https://github.com/awe777/keras-yolo3/blob/master/train.py, dilakukan di cell training di bawah
# '''
print()

Input shape: (None, None, None, 3)
Branch split from main branch - following branch 0
Branch split from branch 0 - following branch 0,0
Branch merge from branch 1 and branch 0,0
Model output 1 shape: (None, None, None, 255)

Branch split from branch 0 - following branch 0,1
Model output 0 shape: (None, None, None, 255)

Weight load attempt success for model_2
Loaded model is successfully re-saved
Model model_2 compilation complete



In [13]:
# model_3 follows float32 ranges, but rounds fractions to 8 bit representation

In [14]:
model_3 = None
'''
model_3_input = Input(shape=(None, None, 3), name="model_3_inputLayer")
# model_3_pointer = model_3_input
print("Input shape:", model_3_input.shape) # 448 x 448 x 3
model_3_startBranch = DBL(roundingFunction=RoundQx_8minusx, previousLayer=model_3_input, layerFilter=16, name="model_3_layer0_branch") 
# model_3_pointer = model_3_startBranch
# print(model_3_pointer.shape) # 448 x 448 x 16
model_3_startBranch = MaxPooling2D(pool_size=(2, 2))(model_3_startBranch)
# model_3_pointer = model_3_startBranch
# print(model_3_pointer.shape) # 224 x 224 x 16
model_3_startBranch = DBL(roundingFunction=RoundQx_8minusx, previousLayer=model_3_startBranch, layerFilter=32, name="model_3_layer1_branch")
# model_3_pointer = model_3_startBranch
# print(model_3_pointer.shape) # 224 x 224 x 32
model_3_startBranch = MaxPooling2D(pool_size=(2, 2))(model_3_startBranch)
# model_3_pointer = model_3_startBranch
# print(model_3_pointer.shape) # 112 x 112 x 32
model_3_startBranch = DBL(roundingFunction=RoundQx_8minusx, previousLayer=model_3_startBranch, layerFilter=64, name="model_3_layer2_branch")
# model_3_pointer = model_3_startBranch
# print(model_3_pointer.shape) # 112 x 112 x 64
model_3_startBranch = MaxPooling2D(pool_size=(2, 2))(model_3_startBranch)
# model_3_pointer = model_3_startBranch
# print(model_3_pointer.shape) # 56 x 56 x 64
model_3_startBranch = DBL(roundingFunction=RoundQx_8minusx, previousLayer=model_3_startBranch, layerFilter=128, name="model_3_layer3_branch")
# model_3_pointer = model_3_startBranch
# print(model_3_pointer.shape) # 56 x 56 x 128
model_3_startBranch = MaxPooling2D(pool_size=(2, 2))(model_3_startBranch)
# model_3_pointer = model_3_startBranch
# print(model_3_pointer.shape) # 28 x 28 x 128
model_3_startBranch = DBL(roundingFunction=RoundQx_8minusx, previousLayer=model_3_startBranch, layerFilter=256, name="model_3_layer4_branch")
# model_3_pointer = model_3_startBranch
# print(model_3_pointer.shape) # 28 x 28 x 256
print("Branch split from main branch - following branch 0") # 2 branch split from startBranch (28 x 28 x 256), following model_3_branch0
model_3_branch0 = MaxPooling2D(pool_size=(2, 2))(model_3_startBranch)
# model_3_pointer = model_3_branch0
# print(model_3_pointer.shape) # 14 x 14 x 256
model_3_branch0 = DBL(roundingFunction=RoundQx_8minusx, previousLayer=model_3_branch0, layerFilter=512, name="model_3_layer5_branch0")
# model_3_pointer = model_3_branch0
# print(model_3_pointer.shape) # 14 x 14 x 512
model_3_branch0 = MaxPooling2D(pool_size=(2, 2), strides=1, padding='same')(model_3_branch0)
# model_3_pointer = model_3_branch0
# print(model_3_pointer.shape) # 14 x 14 x 512
model_3_branch0 = DBL(roundingFunction=RoundQx_8minusx, previousLayer=model_3_branch0, layerFilter=1024, name="model_3_layer6_branch0")
# model_3_pointer = model_3_branch0
# print(model_3_pointer.shape) # 14 x 14 x 1024
model_3_branch0 = DBL(roundingFunction=RoundQx_8minusx, previousLayer=model_3_branch0, layerFilter=256, kernelSize=(1, 1), name="model_3_layer7_branch0")
# model_3_pointer = model_3_branch0
# print(model_3_pointer.shape) # 14 x 14 x 256
print("Branch split from branch 0 - following branch 0,0") # 2 branch split from model_3_branch0 (14 x 14 x 256), following model_3_branch00
model_3_branch00 = DBL(roundingFunction=RoundQx_8minusx, previousLayer=model_3_branch0, layerFilter=128, kernelSize=(1, 1), name="model_3_layer8_branch00")
# model_3_pointer = model_3_branch00
# print(model_3_pointer.shape) # 14 x 14 x 128
model_3_branch00 = UpSampling2D()(model_3_branch00)
# model_3_pointer = model_3_branch00
# print(model_3_pointer.shape) # 28 x 28 x 128
print("Branch merge from branch 1 and branch 0,0") # 2 branch merge from model_3_branch1 (unchanged from model_3_startBranch) and model_3_branch00
model_3_mergedBranch = Concatenate()([model_3_startBranch, model_3_branch00])
# model_3_pointer = model_3_mergedBranch
# print(model_3_pointer.shape) # 28 x 28 x 384
model_3_mergedBranch = DBL(roundingFunction=RoundQx_8minusx, previousLayer=model_3_mergedBranch, layerFilter=256, name="model_3_layer9_branch1")
# model_3_pointer = model_3_mergedBranch
# print(model_3_pointer.shape) # 28 x 28 x 256
model_3_mergedBranch = DBL(roundingFunction=RoundQx_8minusx, previousLayer=model_3_mergedBranch, layerFilter=3 * (4 + 1 + classificationClass), name="model_3_layerA_branch1")
# model_3_pointer = model_3_mergedBranch
print("Model output 1 shape:", model_3_mergedBranch.shape) # 28 x 28 x (3 * (5 + classificationClass))
model_3_mergedBranch = IdentityFinalLayer(name="model_3_outputLayer_1")(model_3_mergedBranch)
print() # OUTPUT = model_3_mergedBranch (note: 26 x 26 grid untuk deteksi objek kecil)

print("Branch split from branch 0 - following branch 0,1")# following model_3_branch01
model_3_branch01 = DBL(roundingFunction=RoundQx_8minusx, previousLayer=model_3_branch0, layerFilter=512, name="model_3_layer8_branch01")
# model_3_pointer = model_3_branch01
# print(model_3_pointer.shape) # 14 x 14 x 512
model_3_branch01 = DBL(roundingFunction=RoundQx_8minusx, previousLayer=model_3_branch01, layerFilter=3 * (4 + 1 + classificationClass), name="model_3_layer9_branch01")
# model_3_pointer = model_3_branch01
print("Model output 0 shape:", model_3_branch01.shape) # 14 x 14 x (3 * (5 + classificationClass))
model_3_branch01 = IdentityFinalLayer(name="model_3_outputLayer_0")(model_3_branch01)
print() # OUTPUT = model_3_branch01 (note: 13 x 13 grid untuk deteksi objek besar)

model_3_actual = Model(inputs=model_3_input, outputs=[model_3_branch01, model_3_mergedBranch]) # mengikuti model dari https://github.com/awe777/keras-yolo3/blob/master/yolo3/model.py

with open("./saved_models/model_3_inferenceModel.json", "wt") as jsonFile:
    jsonFile.write(model_3_actual.to_json())

try:
#     model_3_actual.load_weights("../keras-yolo3/model_data/yolov3_tiny_roundOverflowQ3_4.h5", by_name=True, skip_mismatch=True)
#     model_3_actual.load_weights("./saved_models/model_3_checkpoint.h5", by_name=True, skip_mismatch=True)
    model_3_actual.load_weights("./saved_models/model_0_trainModel.h5", by_name=False, skip_mismatch=True)
    print("Weight load attempt success for model_3")
except Exception as e:
    print("Failed to load existing model for model_3:", e)
try:
    model_3_actual.save_weights("./saved_models/model_3_trainModel.h5")
    print("Loaded model is successfully re-saved")
except Exception as e:
    print("Failed to save loaded model:", e)
# added code from https://github.com/awe777/keras-yolo3/blob/master/yolo3/model.py
setArgs_model_3 = {
    'anchors': tinyYolo_anchors, 
    'num_classes': classificationClass, 
    'ignore_thresh': 0.2
}

y_true_model_3 = [Input(shape=(image_height//{0:32, 1:16}[l], image_width//{0:32, 1:16}[l], 3, classificationClass + 5)) for l in range(2)]

model_3_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss_model_3', arguments=setArgs_model_3)([*model_3_actual.output, *y_true_model_3])

model_3 = Model([model_3_actual.input, *y_true_model_3], model_3_loss)

# model_3_actual adalah working model yang akan dipakai untuk inference
# y_true adalah layer input yang akan diisikan nilai sebenarnya
# cara training model_3 adalah pendekatan sehingga output model_3 sedekat mungkin dengan 0 (model_3 ≈ model_3_actual - y_true)
model_3_learnRate = 1e-2
model_3.compile(optimizer=Adam(lr=model_3_learnRate), loss={'yolo_loss_model_3': lambda y_true, y_pred: y_pred})

# loss adalah seberapa jauh nilai perbedaan output dengan 0

print("Model model_3 compilation complete") # tinggal porting line 54 sampai 61 dari https://github.com/awe777/keras-yolo3/blob/master/train.py, dilakukan di cell training di bawah
# '''
print()

In [15]:
# model_4 approximates Q3.4 signed fixed point operations with integer rules (overflow = positive -> negative & vice versa) 
# Done by rounding to the nearest 1/16 and capping at [-8, 8) after batch normalization and activation layers

In [16]:
model_4 = None
'''
model_4_input = Input(shape=(None, None, 3), name="model_4_inputLayer")
# model_4_pointer = model_4_input
print("Input shape:", model_4_input.shape) # 448 x 448 x 3
model_4_startBranch = DBL(roundingFunction=RoundOverflowQ3_4, previousLayer=model_4_input, layerFilter=16, name="model_4_layer0_branch") 
# model_4_pointer = model_4_startBranch
# print(model_4_pointer.shape) # 448 x 448 x 16
model_4_startBranch = MaxPooling2D(pool_size=(2, 2))(model_4_startBranch)
# model_4_pointer = model_4_startBranch
# print(model_4_pointer.shape) # 224 x 224 x 16
model_4_startBranch = DBL(roundingFunction=RoundOverflowQ3_4, previousLayer=model_4_startBranch, layerFilter=32, name="model_4_layer1_branch")
# model_4_pointer = model_4_startBranch
# print(model_4_pointer.shape) # 224 x 224 x 32
model_4_startBranch = MaxPooling2D(pool_size=(2, 2))(model_4_startBranch)
# model_4_pointer = model_4_startBranch
# print(model_4_pointer.shape) # 112 x 112 x 32
model_4_startBranch = DBL(roundingFunction=RoundOverflowQ3_4, previousLayer=model_4_startBranch, layerFilter=64, name="model_4_layer2_branch")
# model_4_pointer = model_4_startBranch
# print(model_4_pointer.shape) # 112 x 112 x 64
model_4_startBranch = MaxPooling2D(pool_size=(2, 2))(model_4_startBranch)
# model_4_pointer = model_4_startBranch
# print(model_4_pointer.shape) # 56 x 56 x 64
model_4_startBranch = DBL(roundingFunction=RoundOverflowQ3_4, previousLayer=model_4_startBranch, layerFilter=128, name="model_4_layer3_branch")
# model_4_pointer = model_4_startBranch
# print(model_4_pointer.shape) # 56 x 56 x 128
model_4_startBranch = MaxPooling2D(pool_size=(2, 2))(model_4_startBranch)
# model_4_pointer = model_4_startBranch
# print(model_4_pointer.shape) # 28 x 28 x 128
model_4_startBranch = DBL(roundingFunction=RoundOverflowQ3_4, previousLayer=model_4_startBranch, layerFilter=256, name="model_4_layer4_branch")
# model_4_pointer = model_4_startBranch
# print(model_4_pointer.shape) # 28 x 28 x 256
print("Branch split from main branch - following branch 0") # 2 branch split from startBranch (28 x 28 x 256), following model_4_branch0
model_4_branch0 = MaxPooling2D(pool_size=(2, 2))(model_4_startBranch)
# model_4_pointer = model_4_branch0
# print(model_4_pointer.shape) # 14 x 14 x 256
model_4_branch0 = DBL(roundingFunction=RoundOverflowQ3_4, previousLayer=model_4_branch0, layerFilter=512, name="model_4_layer5_branch0")
# model_4_pointer = model_4_branch0
# print(model_4_pointer.shape) # 14 x 14 x 512
model_4_branch0 = MaxPooling2D(pool_size=(2, 2), strides=1, padding='same')(model_4_branch0)
# model_4_pointer = model_4_branch0
# print(model_4_pointer.shape) # 14 x 14 x 512
model_4_branch0 = DBL(roundingFunction=RoundOverflowQ3_4, previousLayer=model_4_branch0, layerFilter=1024, name="model_4_layer6_branch0")
# model_4_pointer = model_4_branch0
# print(model_4_pointer.shape) # 14 x 14 x 1024
model_4_branch0 = DBL(roundingFunction=RoundOverflowQ3_4, previousLayer=model_4_branch0, layerFilter=256, kernelSize=(1, 1), name="model_4_layer7_branch0")
# model_4_pointer = model_4_branch0
# print(model_4_pointer.shape) # 14 x 14 x 256
print("Branch split from branch 0 - following branch 0,0") # 2 branch split from model_4_branch0 (14 x 14 x 256), following model_4_branch00
model_4_branch00 = DBL(roundingFunction=RoundOverflowQ3_4, previousLayer=model_4_branch0, layerFilter=128, kernelSize=(1, 1), name="model_4_layer8_branch00")
# model_4_pointer = model_4_branch00
# print(model_4_pointer.shape) # 14 x 14 x 128
model_4_branch00 = UpSampling2D()(model_4_branch00)
# model_4_pointer = model_4_branch00
# print(model_4_pointer.shape) # 28 x 28 x 128
print("Branch merge from branch 1 and branch 0,0") # 2 branch merge from model_4_branch1 (unchanged from model_4_startBranch) and model_4_branch00
model_4_mergedBranch = Concatenate()([model_4_startBranch, model_4_branch00])
# model_4_pointer = model_4_mergedBranch
# print(model_4_pointer.shape) # 28 x 28 x 384
model_4_mergedBranch = DBL(roundingFunction=RoundOverflowQ3_4, previousLayer=model_4_mergedBranch, layerFilter=256, name="model_4_layer9_branch1")
# model_4_pointer = model_4_mergedBranch
# print(model_4_pointer.shape) # 28 x 28 x 256
model_4_mergedBranch = DBL(roundingFunction=RoundOverflowQ3_4, previousLayer=model_4_mergedBranch, layerFilter=3 * (4 + 1 + classificationClass), name="model_4_layerA_branch1")
# model_4_pointer = model_4_mergedBranch
print("Model output 1 shape:", model_4_mergedBranch.shape) # 28 x 28 x (3 * (5 + classificationClass))
model_4_mergedBranch = IdentityFinalLayer(name="model_4_outputLayer_1")(model_4_mergedBranch)
print() # OUTPUT = model_4_mergedBranch (note: 26 x 26 grid untuk deteksi objek kecil)

print("Branch split from branch 0 - following branch 0,1")# following model_4_branch01
model_4_branch01 = DBL(roundingFunction=RoundOverflowQ3_4, previousLayer=model_4_branch0, layerFilter=512, name="model_4_layer8_branch01")
# model_4_pointer = model_4_branch01
# print(model_4_pointer.shape) # 14 x 14 x 512
model_4_branch01 = DBL(roundingFunction=RoundOverflowQ3_4, previousLayer=model_4_branch01, layerFilter=3 * (4 + 1 + classificationClass), name="model_4_layer9_branch01")
# model_4_pointer = model_4_branch01
print("Model output 0 shape:", model_4_branch01.shape) # 14 x 14 x (3 * (5 + classificationClass))
model_4_branch01 = IdentityFinalLayer(name="model_4_outputLayer_0")(model_4_branch01)
print() # OUTPUT = model_4_branch01 (note: 13 x 13 grid untuk deteksi objek besar)

model_4_actual = Model(inputs=model_4_input, outputs=[model_4_branch01, model_4_mergedBranch]) # mengikuti model dari https://github.com/awe777/keras-yolo3/blob/master/yolo3/model.py

with open("./saved_models/model_4_inferenceModel.json", "wt") as jsonFile:
    jsonFile.write(model_4_actual.to_json())

try:
#     model_4_actual.load_weights("../keras-yolo3/model_data/yolov3_tiny-model_4.h5", by_name=True, skip_mismatch=True)
    model_4_actual.load_weights("./saved_models/model_4_checkpoint.h5", by_name=True, skip_mismatch=True)
    print("Weight load attempt success for model_4")
except Exception as e:
    print("Failed to load existing model for model_4:", e)
try:
    model_4_actual.save_weights("./saved_models/model_4_trainModel.h5")
    print("Loaded model is successfully re-saved")
except Exception as e:
    print("Failed to save loaded model:", e)
# added code from https://github.com/awe777/keras-yolo3/blob/master/yolo3/model.py
setArgs_model_4 = {
    'anchors': tinyYolo_anchors, 
    'num_classes': classificationClass, 
    'ignore_thresh': 0.2
}

y_true_model_4 = [Input(shape=(image_height//{0:32, 1:16}[l], image_width//{0:32, 1:16}[l], 3, classificationClass + 5)) for l in range(2)]

model_4_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss_model_4', arguments=setArgs_model_4)([*model_4_actual.output, *y_true_model_4])

model_4 = Model([model_4_actual.input, *y_true_model_4], model_4_loss)

# model_4_actual adalah working model yang akan dipakai untuk inference
# y_true adalah layer input yang akan diisikan nilai sebenarnya
# cara training model_4 adalah pendekatan sehingga output model_4 sedekat mungkin dengan 0 (model_4 ≈ model_4_actual - y_true)
model_4_learnRate = 1e-3
model_4.compile(optimizer=Adam(lr=model_4_learnRate), loss={'yolo_loss_model_4': lambda y_true, y_pred: y_pred})

# loss adalah seberapa jauh nilai perbedaan output dengan 0

print("Model model_4 compilation complete") # tinggal porting line 54 sampai 61 dari https://github.com/awe777/keras-yolo3/blob/master/train.py, dilakukan di cell training di bawah
# '''
print()

In [17]:
if model_0 is not None:
    with open("./saved_models/model_0_summary.txt", "wt") as textFile:
        model_0.summary(print_fn=lambda x: textFile.write(x + '\n'), line_length=200)

In [18]:
if model_1 is not None:
    with open("./saved_models/model_1_summary.txt", "wt") as textFile:
        model_1.summary(print_fn=lambda x: textFile.write(x + '\n'), line_length=200)

In [19]:
if model_2 is not None:
    with open("./saved_models/model_2_summary.txt", "wt") as textFile:
        model_2.summary(print_fn=lambda x: textFile.write(x + '\n'), line_length=200)

In [20]:
if model_3 is not None:
    with open("./saved_models/model_3_summary.txt", "wt") as textFile:
        model_3.summary(print_fn=lambda x: textFile.write(x + '\n'), line_length=200)

In [21]:
if model_4 is not None:
    with open("./saved_models/model_4_summary.txt", "wt") as textFile:
        model_4.summary(print_fn=lambda x: textFile.write(x + '\n'), line_length=200)

In [22]:
print()
trainingBatchSize = 1
epochSplit = 1047 
# because high split = more work saved; 117266 mod 499 = 1 <= less image lost for (416,416)
# 117266 mod 1047 = 2 <= due to high loss of (448,448)
train_data_generator = data_generator_wrapper(
    annotation_lines=train_annotation_lines, 
    batch_size=trainingBatchSize, 
    input_shape=image_size, 
    anchors=tinyYolo_anchors, 
    num_classes=classificationClass
)
val_data_generator = data_generator_wrapper(
    annotation_lines=val_annotation_lines, 
    batch_size=trainingBatchSize, 
    input_shape=image_size, 
    anchors=tinyYolo_anchors, 
    num_classes=classificationClass
)
minimumLR = 1e-4
decayChance = 0.50
class BestValueRecorder(Callback):
    def __init__(self, filepath, monitorValidation=False, mode='min', verbose=1, instanceModelCheckpointLoss=None, instanceModelCheckpointVal=None):
        super(BestValueRecorder, self).__init__()
        self.lookOnVal = monitorValidation
        self.verbose = verbose
        if mode not in ['min', 'max']:
            mode = 'min'
        if mode == 'min':
            self.monitor_op = np.less
            self.MCLossValue = np.Inf
            self.MCValLossValue = np.Inf
        else:
            self.monitor_op = np.greater
            self.MCLossValue = -np.Inf
            self.MCValLossValue = -np.Inf
        self.filepath = filepath
        try:
            with open(self.filepath, 'rt') as jsonFile:
                bestValueList = json.loads(jsonFile.read())
                self.MCLossValue =  bestValueList[0]
                if self.lookOnVal:
                    self.MCValLossValue = bestValueList[1]
        except Exception as e:
            print("Failed to open JSON file:", e)
        if instanceModelCheckpointLoss is not None:
            instanceModelCheckpointLoss.best = self.MCLossValue
        if instanceModelCheckpointVal is not None and monitorValidation:
            instanceModelCheckpointVal.best = self.MCValLossValue

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        currentLoss = logs.get('loss')
        newChange = False
        if currentLoss is not None and self.monitor_op(currentLoss, self.MCLossValue):
            self.MCLossValue = currentLoss
            newChange = True
        if self.lookOnVal:
            currentValLoss = logs.get('val_loss')
            if currentValLoss is not None and self.monitor_op(currentValLoss, self.MCValLossValue):
                self.MCValLossValue = currentValLoss
                newChange = True
        if newChange:
            try:
                with open(self.filepath, 'wt') as jsonFile:
                    if self.verbose > 0:
                        print("Saving best loss value:", [self.MCLossValue, self.MCValLossValue])
                    jsonFile.write(json.dumps([self.MCLossValue, self.MCValLossValue]))
            except Exception as e:
                print("Failed to open JSON file:", e)
print()

In [23]:
print()
''' // comment on this line to enable/disable this block
model_0_checkpoint_val = ModelCheckpoint(
    filepath='./saved_models/model_0_checkpoint.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='min'
)
model_0_checkpoint_loss = ModelCheckpoint(
    filepath='./saved_models/model_0_checkpoint.h5',
    monitor='loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='min',
    period=2
)
model_0_recorder = BestValueRecorder(
    filepath='./saved_models/model_0_bestValue.json',
    monitorValidation=True,
    verbose=1,
    mode='min',
    # these instances are modified by the __init__ (constructor) of this object
    instanceModelCheckpointLoss=model_0_checkpoint_loss,
    instanceModelCheckpointVal=model_0_checkpoint_val
)
model_0_ER = EarlyStopping(
    monitor='loss',
    patience=15,
    verbose=1,
    mode='min',
    baseline=model_0_checkpoint_loss.best # this instance is not modified by the __init__ of this object
)
model_0_LRDecay = math.pow(1 / 4, 1 / 3) # exponentially decays to 25% in 3 super-epochs 
superEpochs = 0
start_time = time.time()
while time.time() - start_time < 6 * 3600: # guarantees at least 6 hours of training, unless the kernel crashes
# while superEpochs <= 20: # guarantees 10 super-epochs, unless the kernel crashes
    superEpochs = superEpochs + 1
    print('Time', str(time.time() - start_time))
    print('Super-epoch', superEpochs, '- learn rate:', model_0_learnRate, '- minimum LR:', minimumLR)
    print()
    model_0_history = model_0.fit_generator(
        generator=train_data_generator, 
        steps_per_epoch=max(1, (lenTrain // trainingBatchSize) // epochSplit), 
        epochs=1 * epochSplit,
        verbose=2,
        validation_data=val_data_generator,
        validation_steps=max(1, 15 * (lenVal // trainingBatchSize) // epochSplit),
        callbacks=[ 
            TerminateOnNaN(),
            model_0_checkpoint_val,
            model_0_checkpoint_loss,
            model_0_recorder,
            model_0_ER
        ]
    )
    if np.random.rand() >= decayChance:
        model_0_ER.patience = model_0_ER.patience * 1.15
        print("Early stopping patience rate increased -", model_0_ER.patience)
    if model_0_learnRate * model_0_LRDecay >= minimumLR and np.random.rand() < decayChance:
        # guarantees learn rate above minimumLR
        # randomly decays learnRate with a predetermined probability
        print("Learn rate decayed")
        model_0_learnRate = model_0_LRDecay * model_0_learnRate
    model_0.load_weights("./saved_models/model_0_checkpoint.h5")
    model_0.compile(optimizer=Adam(lr=model_0_learnRate), loss={'yolo_loss_model_0': lambda y_true, y_pred: y_pred})
    print()
print("model_0 training done in", str(time.time() - start_time))
model_0.save_weights("./saved_models/model_0_trainModel.h5")
# '''
print()

In [24]:
print()
# ''' // comment on this line to enable/disable this block
model_1_checkpoint_val = ModelCheckpoint(
    filepath='./saved_models/model_1_checkpoint.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='min'
)
model_1_checkpoint_loss = ModelCheckpoint(
    filepath='./saved_models/model_1_checkpoint.h5',
    monitor='loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='min',
    period=2
)
model_1_recorder = BestValueRecorder(
    filepath='./saved_models/model_1_bestValue.json',
    monitorValidation=True,
    verbose=1,
    mode='min',
    # these instances are modified by the __init__ (constructor) of this object
    instanceModelCheckpointLoss=model_1_checkpoint_loss,
    instanceModelCheckpointVal=model_1_checkpoint_val
)
model_1_ER = EarlyStopping(
    monitor='loss',
    patience=15,
    verbose=1,
    mode='min',
    baseline=model_1_checkpoint_loss.best # this instance is not modified by the __init__ of this object
)
model_1_LRDecay = math.pow(1 / 4, 1 / 3) # exponentially decays to 25% in 3 super-epochs 
superEpochs = 0
start_time = time.time()
while time.time() - start_time < 1 * 3600: # guarantees at least 6 hours of training, unless the kernel crashes
# while superEpochs <= 20: # guarantees 10 super-epochs, unless the kernel crashes
    superEpochs = superEpochs + 1
    print('Time', str(time.time() - start_time))
    print('Super-epoch', superEpochs, '- learn rate:', model_1_learnRate, '- minimum LR:', minimumLR)
    print()
    model_1_history = model_1.fit_generator(
        generator=train_data_generator, 
        steps_per_epoch=max(1, (lenTrain // trainingBatchSize) // epochSplit), 
        epochs=1 * epochSplit,
        verbose=2,
        validation_data=val_data_generator,
        validation_steps=max(1, 15 * (lenVal // trainingBatchSize) // epochSplit),
        callbacks=[ 
            TerminateOnNaN(),
            model_1_checkpoint_val,
            model_1_checkpoint_loss,
            model_1_recorder,
            model_1_ER
        ]
    )
    if np.random.rand() >= decayChance:
        model_1_ER.patience = model_1_ER.patience * 1.15
        print("Early stopping patience rate increased -", model_1_ER.patience)
    if model_1_learnRate * model_1_LRDecay >= minimumLR and np.random.rand() < decayChance:
        # guarantees learn rate above minimumLR
        # randomly decays learnRate with a predetermined probability
        print("Learn rate decayed")
        model_1_learnRate = model_1_LRDecay * model_1_learnRate
    model_1.load_weights("./saved_models/model_1_checkpoint.h5")
    model_1.compile(optimizer=Adam(lr=model_1_learnRate), loss={'yolo_loss_model_1': lambda y_true, y_pred: y_pred})
    print()
print("model_1 training done in", str(time.time() - start_time))
model_1.save_weights("./saved_models/model_1_trainModel.h5")
# '''
print()

In [25]:
print()
# ''' // comment on this line to enable/disable this block
model_2_checkpoint_val = ModelCheckpoint(
    filepath='./saved_models/model_2_checkpoint.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='min'
)
model_2_checkpoint_loss = ModelCheckpoint(
    filepath='./saved_models/model_2_checkpoint.h5',
    monitor='loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='min',
    period=2
)
model_2_recorder = BestValueRecorder(
    filepath='./saved_models/model_2_bestValue.json',
    monitorValidation=True,
    verbose=1,
    mode='min',
    # these instances are modified by the __init__ (constructor) of this object
    instanceModelCheckpointLoss=model_2_checkpoint_loss,
    instanceModelCheckpointVal=model_2_checkpoint_val
)
model_2_ER = EarlyStopping(
    monitor='loss',
    patience=15,
    verbose=1,
    mode='min',
    baseline=model_2_checkpoint_loss.best # this instance is not modified by the __init__ of this object
)
model_2_LRDecay = math.pow(1 / 4, 1 / 3) # exponentially decays to 25% in 3 super-epochs 
superEpochs = 0
start_time = time.time()
while time.time() - start_time < 6 * 3600: # guarantees at least 6 hours of training, unless the kernel crashes
# while superEpochs <= 20: # guarantees 10 super-epochs, unless the kernel crashes
    superEpochs = superEpochs + 1
    print('Time', str(time.time() - start_time))
    print('Super-epoch', superEpochs, '- learn rate:', model_2_learnRate, '- minimum LR:', minimumLR)
    print()
    model_2_history = model_2.fit_generator(
        generator=train_data_generator, 
        steps_per_epoch=max(1, (lenTrain // trainingBatchSize) // epochSplit), 
        epochs=1 * epochSplit,
        verbose=2,
        validation_data=val_data_generator,
        validation_steps=max(1, 15 * (lenVal // trainingBatchSize) // epochSplit),
        callbacks=[ 
            TerminateOnNaN(),
            model_2_checkpoint_val,
            model_2_checkpoint_loss,
            model_2_recorder,
            model_2_ER
        ]
    )
    if np.random.rand() >= decayChance:
        model_2_ER.patience = model_2_ER.patience * 1.15
        print("Early stopping patience rate increased -", model_2_ER.patience)
    if model_2_learnRate * model_2_LRDecay >= minimumLR and np.random.rand() < decayChance:
        # guarantees learn rate above minimumLR
        # randomly decays learnRate with a predetermined probability
        print("Learn rate decayed")
        model_2_learnRate = model_2_LRDecay * model_2_learnRate
    model_2.load_weights("./saved_models/model_2_checkpoint.h5")
    model_2.compile(optimizer=Adam(lr=model_2_learnRate), loss={'yolo_loss_model_2': lambda y_true, y_pred: y_pred})
    print()
print("model_2 training done in", str(time.time() - start_time))
model_2.save_weights("./saved_models/model_2_trainModel.h5")
# '''
print()


Failed to open JSON file: [Errno 2] No such file or directory: './saved_models/model_2_bestValue.json'
Time 0.0
Super-epoch 1 - learn rate: 0.01 - minimum LR: 0.0001

Epoch 1/1047
 - 84s - loss: 2531.2998 - val_loss: 2042.8627

Epoch 00001: val_loss improved from inf to 2042.86267, saving model to ./saved_models/model_2_checkpoint.h5
Saving best loss value: [2531.299815586635, 2042.8626708984375]
Epoch 2/1047
 - 72s - loss: 2225.5097 - val_loss: 2163.5730

Epoch 00002: val_loss did not improve from 2042.86267

Epoch 00002: loss improved from inf to 2225.50969, saving model to ./saved_models/model_2_checkpoint.h5
Saving best loss value: [2225.509690420968, 2042.8626708984375]
Epoch 3/1047
 - 71s - loss: 2227.3109 - val_loss: 2022.4705

Epoch 00003: val_loss improved from 2042.86267 to 2022.47046, saving model to ./saved_models/model_2_checkpoint.h5
Saving best loss value: [2225.509690420968, 2022.470458984375]
Epoch 4/1047
 - 71s - loss: 2257.8190 - val_loss: 2095.5833

Epoch 00004: va


Epoch 00051: val_loss did not improve from 2005.28320
Epoch 52/1047
 - 69s - loss: 2257.2846 - val_loss: 2231.5259

Epoch 00052: val_loss did not improve from 2005.28320

Epoch 00052: loss did not improve from 2208.03608
Epoch 53/1047
 - 69s - loss: 2226.6935 - val_loss: 3083.0083

Epoch 00053: val_loss did not improve from 2005.28320
Epoch 54/1047
 - 69s - loss: 2229.8736 - val_loss: 2336.9849

Epoch 00054: val_loss did not improve from 2005.28320

Epoch 00054: loss did not improve from 2208.03608
Epoch 55/1047
 - 69s - loss: 2212.7194 - val_loss: 2149.4700

Epoch 00055: val_loss did not improve from 2005.28320
Epoch 56/1047
 - 69s - loss: 2263.8321 - val_loss: 2689.0605

Epoch 00056: val_loss did not improve from 2005.28320

Epoch 00056: loss did not improve from 2208.03608
Epoch 57/1047
 - 69s - loss: 2265.2279 - val_loss: 1990.6191

Epoch 00057: val_loss improved from 2005.28320 to 1990.61914, saving model to ./saved_models/model_2_checkpoint.h5
Saving best loss value: [2205.45223

 - 70s - loss: 2239.2454 - val_loss: 2015.4130

Epoch 00025: val_loss did not improve from 1990.61914

Epoch 00025: loss did not improve from 2192.08419
Epoch 26/1047
 - 69s - loss: 2271.5136 - val_loss: 2109.2097

Epoch 00026: val_loss did not improve from 1990.61914
Epoch 27/1047
 - 69s - loss: 2252.5339 - val_loss: 2171.6279

Epoch 00027: val_loss did not improve from 1990.61914

Epoch 00027: loss did not improve from 2192.08419
Epoch 28/1047
 - 69s - loss: 2247.9505 - val_loss: 2048.8162

Epoch 00028: val_loss did not improve from 1990.61914
Epoch 29/1047
 - 70s - loss: 2251.3152 - val_loss: 2297.7605

Epoch 00029: val_loss did not improve from 1990.61914

Epoch 00029: loss did not improve from 2192.08419
Epoch 30/1047
 - 69s - loss: 2206.8645 - val_loss: 2284.2363

Epoch 00030: val_loss did not improve from 1990.61914
Epoch 31/1047
 - 69s - loss: 2272.6812 - val_loss: 2090.4595

Epoch 00031: val_loss did not improve from 1990.61914

Epoch 00031: loss did not improve from 2192.0841

 - 69s - loss: 2231.4338 - val_loss: 2211.1531

Epoch 00042: val_loss did not improve from 1990.61914
Epoch 43/1047
 - 69s - loss: 2203.1212 - val_loss: 2093.3423

Epoch 00043: val_loss did not improve from 1990.61914

Epoch 00043: loss did not improve from 2192.08419
Epoch 44/1047
 - 69s - loss: 2239.7888 - val_loss: 2021.5103

Epoch 00044: val_loss did not improve from 1990.61914
Epoch 45/1047
 - 69s - loss: 2272.9632 - val_loss: 2551.2080

Epoch 00045: val_loss did not improve from 1990.61914

Epoch 00045: loss did not improve from 2192.08419
Epoch 46/1047
 - 69s - loss: 2245.2685 - val_loss: 2240.0513

Epoch 00046: val_loss did not improve from 1990.61914
Epoch 47/1047
 - 69s - loss: 2252.4626 - val_loss: 2330.5461

Epoch 00047: val_loss did not improve from 1990.61914

Epoch 00047: loss did not improve from 2192.08419
Epoch 48/1047
 - 69s - loss: 2212.9644 - val_loss: 2042.9124

Epoch 00048: val_loss did not improve from 1990.61914
Epoch 00048: early stopping
Early stopping patien

 - 69s - loss: 2227.1633 - val_loss: 1990.5807

Epoch 00012: val_loss improved from 1990.61914 to 1990.58069, saving model to ./saved_models/model_2_checkpoint.h5
Saving best loss value: [2192.0841936383927, 1990.5806884765625]
Epoch 13/1047
 - 69s - loss: 2242.6655 - val_loss: 1984.7996

Epoch 00013: val_loss improved from 1990.58069 to 1984.79956, saving model to ./saved_models/model_2_checkpoint.h5

Epoch 00013: loss did not improve from 2192.08419
Saving best loss value: [2192.0841936383927, 1984.799560546875]
Epoch 14/1047
 - 69s - loss: 2252.0465 - val_loss: 2151.4705

Epoch 00014: val_loss did not improve from 1984.79956
Epoch 15/1047
 - 69s - loss: 2238.9888 - val_loss: 2203.8669

Epoch 00015: val_loss did not improve from 1984.79956

Epoch 00015: loss did not improve from 2192.08419
Epoch 16/1047
 - 69s - loss: 2255.5341 - val_loss: 2364.9297

Epoch 00016: val_loss did not improve from 1984.79956
Epoch 17/1047
 - 69s - loss: 2214.9057 - val_loss: 2567.7854

Epoch 00017: val_lo

 - 68s - loss: 2264.2254 - val_loss: 2014.1459

Epoch 00013: val_loss did not improve from 1984.79956

Epoch 00013: loss did not improve from 2187.28843
Epoch 14/1047
 - 68s - loss: 2270.7809 - val_loss: 2429.1375

Epoch 00014: val_loss did not improve from 1984.79956
Epoch 15/1047
 - 68s - loss: 2274.7318 - val_loss: 2041.4683

Epoch 00015: val_loss did not improve from 1984.79956

Epoch 00015: loss did not improve from 2187.28843
Epoch 16/1047
 - 68s - loss: 2252.4823 - val_loss: 1983.5275

Epoch 00016: val_loss improved from 1984.79956 to 1983.52747, saving model to ./saved_models/model_2_checkpoint.h5
Saving best loss value: [2187.288433619908, 1983.5274658203125]
Epoch 17/1047
 - 69s - loss: 2264.1311 - val_loss: 2013.4421

Epoch 00017: val_loss did not improve from 1983.52747

Epoch 00017: loss did not improve from 2187.28843
Epoch 18/1047
 - 68s - loss: 2222.2691 - val_loss: 2314.6904

Epoch 00018: val_loss did not improve from 1983.52747
Epoch 19/1047
 - 68s - loss: 2228.1134 -


Epoch 00069: val_loss did not improve from 1983.52747

Epoch 00069: loss did not improve from 2171.09154
Epoch 70/1047
 - 69s - loss: 2253.4793 - val_loss: 2380.1780

Epoch 00070: val_loss did not improve from 1983.52747
Epoch 71/1047
 - 68s - loss: 2274.4039 - val_loss: 2240.1084

Epoch 00071: val_loss did not improve from 1983.52747

Epoch 00071: loss did not improve from 2171.09154
Epoch 72/1047
 - 68s - loss: 2226.7882 - val_loss: 2264.6482

Epoch 00072: val_loss did not improve from 1983.52747
Epoch 73/1047
 - 68s - loss: 2277.9852 - val_loss: 2195.1189

Epoch 00073: val_loss did not improve from 1983.52747

Epoch 00073: loss did not improve from 2171.09154
Epoch 74/1047
 - 68s - loss: 2232.5369 - val_loss: 2196.5217

Epoch 00074: val_loss did not improve from 1983.52747
Epoch 00074: early stopping
Learn rate decayed

model_2 training done in 23139.20682477951



In [26]:
print()
''' // comment on this line to enable/disable this block
model_3_checkpoint_val = ModelCheckpoint(
    filepath='./saved_models/model_3_checkpoint.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='min'
)
model_3_checkpoint_loss = ModelCheckpoint(
    filepath='./saved_models/model_3_checkpoint.h5',
    monitor='loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='min',
    period=2
)
model_3_recorder = BestValueRecorder(
    filepath='./saved_models/model_3_bestValue.json',
    monitorValidation=True,
    verbose=1,
    mode='min',
    # these instances are modified by the __init__ (constructor) of this object
    instanceModelCheckpointLoss=model_3_checkpoint_loss,
    instanceModelCheckpointVal=model_3_checkpoint_val
)
model_3_ER = EarlyStopping(
    monitor='loss',
    patience=15,
    verbose=1,
    mode='min',
    baseline=model_3_checkpoint_loss.best # this instance is not modified by the __init__ of this object
)
model_3_LRDecay = math.pow(1 / 4, 1 / 3) # exponentially decays to 25% in 3 super-epochs 
superEpochs = 0
start_time = time.time()
while time.time() - start_time < 6 * 3600: # guarantees at least 6 hours of training, unless the kernel crashes
# while superEpochs <= 20: # guarantees 10 super-epochs, unless the kernel crashes
    superEpochs = superEpochs + 1
    print('Time', str(time.time() - start_time))
    print('Super-epoch', superEpochs, '- learn rate:', model_3_learnRate, '- minimum LR:', minimumLR)
    print()
    model_3_history = model_3.fit_generator(
        generator=train_data_generator, 
        steps_per_epoch=max(1, (lenTrain // trainingBatchSize) // epochSplit), 
        epochs=1 * epochSplit,
        verbose=2,
        validation_data=val_data_generator,
        validation_steps=max(1, 15 * (lenVal // trainingBatchSize) // epochSplit),
        callbacks=[ 
            TerminateOnNaN(),
            model_3_checkpoint_val,
            model_3_checkpoint_loss,
            model_3_recorder,
            model_3_ER
        ]
    )
    if np.random.rand() >= decayChance:
        model_3_ER.patience = model_3_ER.patience * 1.15
        print("Early stopping patience rate increased -", model_3_ER.patience)
    if model_3_learnRate * model_3_LRDecay >= minimumLR and np.random.rand() < decayChance:
        # guarantees learn rate above minimumLR
        # randomly decays learnRate with a predetermined probability
        print("Learn rate decayed")
        model_3_learnRate = model_3_LRDecay * model_3_learnRate
    model_3.load_weights("./saved_models/model_3_checkpoint.h5")
    model_3.compile(optimizer=Adam(lr=model_3_learnRate), loss={'yolo_loss_model_3': lambda y_true, y_pred: y_pred})
    print()
print("model_3 training done in", str(time.time() - start_time))
model_3.save_weights("./saved_models/model_3_trainModel.h5")
# '''
print()

In [27]:
print()
''' // comment on this line to enable/disable this block
model_4_checkpoint_val = ModelCheckpoint(
    filepath='./saved_models/model_4_checkpoint.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='min'
)
model_4_checkpoint_loss = ModelCheckpoint(
    filepath='./saved_models/model_4_checkpoint.h5',
    monitor='loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='min',
    period=2
)
model_4_recorder = BestValueRecorder(
    filepath='./saved_models/model_4_bestValue.json',
    monitorValidation=True,
    verbose=1,
    mode='min',
    # these instances are modified by the __init__ (constructor) of this object
    instanceModelCheckpointLoss=model_4_checkpoint_loss,
    instanceModelCheckpointVal=model_4_checkpoint_val
)
model_4_ER = EarlyStopping(
    monitor='loss',
    patience=15,
    verbose=1,
    mode='min',
    baseline=model_4_checkpoint_loss.best # this instance is not modified by the __init__ of this object
)
model_4_LRDecay = math.pow(1 / 4, 1 / 8) # exponentially decays to 25% in 3 super-epochs 
superEpochs = 0
start_time = time.time()
while time.time() - start_time < 6 * 3600: # guarantees at least 6 hours of training, unless the kernel crashes
# while superEpochs <= 20: # guarantees 10 super-epochs, unless the kernel crashes
    superEpochs = superEpochs + 1
    print('Time', str(time.time() - start_time))
    print('Super-epoch', superEpochs, '- learn rate:', model_4_learnRate, '- minimum LR:', minimumLR)
    print()
    model_4_history = model_4.fit_generator(
        generator=train_data_generator, 
        steps_per_epoch=max(1, (lenTrain // trainingBatchSize) // epochSplit), 
        epochs=1 * epochSplit,
        verbose=2,
        validation_data=val_data_generator,
        validation_steps=max(1, 15 * (lenVal // trainingBatchSize) // epochSplit),
        callbacks=[ 
            TerminateOnNaN(),
            model_4_checkpoint_val,
            model_4_checkpoint_loss,
            model_4_recorder,
            model_4_ER
        ]
    )
    if np.random.rand() >= decayChance:
        model_4_ER.patience = model_4_ER.patience * 1.15
        print("Early stopping patience rate increased -", model_4_ER.patience)
    if model_4_learnRate * model_4_LRDecay >= minimumLR and np.random.rand() < decayChance:
        # guarantees learn rate above minimumLR
        # randomly decays learnRate with a predetermined probability
        print("Learn rate decayed")
        model_4_learnRate = model_4_LRDecay * model_4_learnRate
    model_4.load_weights("./saved_models/model_4_checkpoint.h5")
    model_4.compile(optimizer=Adam(lr=model_4_learnRate), loss={'yolo_loss_model_4': lambda y_true, y_pred: y_pred})
    print()
print("model_4 training done in", str(time.time() - start_time))
model_4.save_weights("./saved_models/model_4_trainModel.h5")
# '''
print()

In [28]:
print("Resource successfully released")

Resource successfully released
